<a href="https://colab.research.google.com/github/walkerjian/DailyCode/blob/main/Code_Craft_HeapStackWithC_too.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Problem:
Implement a stack API using only a heap. A stack implements the following methods:

push(item), which adds an element to the stack
pop(), which removes and returns the most recently added element (or throws an error if there is nothing on the stack)
Recall that a heap has the following operations:

push(item), which adds a new key to the heap
pop(), which removes and returns the max value of the heap

##Solution:
Implementing a stack API using only a heap requires a strategy to maintain the stack's Last In, First Out (LIFO) order using the heap's structure, which naturally follows a different order, either Min Heap (the smallest element is always on top) or Max Heap (the largest element is always on top).

The core challenge is that a heap is designed to efficiently remove the smallest or largest element (depending on if it's a min-heap or max-heap), not the most recently added element. To simulate stack behavior (LIFO) with a heap, we can use timestamps or an increasing counter to keep track of the insertion order. By associating each element pushed onto the "stack" with an incrementing count, we can ensure that the most recently pushed item can be identified as having the highest count. Thus, by using a max-heap based on these counts, the item most recently added will always be at the top of the heap.

Here's a Python class that implements a stack using a max heap. We'll use the heapq module, which provides an implementation of the min heap. To create a max heap, we'll store tuples in the heap with a count that decreases with each push (as Python's heapq is a min heap, we invert the count to simulate max heap behavior):

```python
import heapq

class StackUsingHeap:
    def __init__(self):
        self.heap = []
        self.current = 0  # This will act as a decreasing counter
    
    def push(self, item):
        # Use a negative count because heapq is a min heap, but we need max heap behavior
        heapq.heappush(self.heap, (-self.current, item))
        self.current += 1  # Increment the counter for the next item
    
    def pop(self):
        if not self.heap:
            raise IndexError("pop from an empty stack")
        # heapq.heappop returns the smallest item, so use -count to simulate stack behavior
        _, item = heapq.heappop(self.heap)
        return item

# Example usage
stack = StackUsingHeap()
stack.push(1)
stack.push(2)
stack.push(3)

print(stack.pop())  # Should print 3
print(stack.pop())  # Should print 2
print(stack.pop())  # Should print 1
```

This implementation leverages a decreasing counter to ensure that the most recently added item will always be considered the "largest" in the context of the min heap provided by Python's `heapq` module. The `push` method adds elements with their count to the heap, and the `pop` method retrieves and removes the element with the highest count, effectively mimicking stack behavior.

##Testing:

In [1]:
import heapq

class StackUsingHeap:
    def __init__(self):
        self.heap = []
        self.current = 0  # This will act as a decreasing counter

    def push(self, item):
        # Use a negative count because heapq is a min heap, but we need max heap behavior
        heapq.heappush(self.heap, (-self.current, item))
        self.current += 1  # Increment the counter for the next item

    def pop(self):
        if not self.heap:
            raise IndexError("pop from an empty stack")
        # heapq.heappop returns the smallest item, so use -count to simulate stack behavior
        _, item = heapq.heappop(self.heap)
        return item

# Example usage
stack = StackUsingHeap()
stack.push(1)
stack.push(2)
stack.push(3)

print(stack.pop())  # Should print 3
print(stack.pop())  # Should print 2
print(stack.pop())  # Should print 1


3
2
1


##Implementing a stack in a heap with c:
Implementing a stack using a heap in C requires manually managing the heap's memory and implementing the heap operations to maintain the stack's Last In, First Out (LIFO) behavior. We'll create a max heap where each element is a struct containing the item and its insertion order (a timestamp or counter). Since C does not have a built-in heap library like Python, we'll need to implement both the heap and the stack functionalities from scratch.

Here's an outline of how to do it:

1. **Define a Struct for Heap Elements**: Each element will have an integer for ordering (`order`) and a generic pointer (`void*`) to hold the stack item. Using `void*` allows the stack to store any type of data.

2. **Heap Operations**: Implement basic heap operations such as `heap_push`, `heap_pop`, and any helper functions needed for maintaining the heap property (e.g., `heapify` or `siftUp` and `siftDown`).

3. **Stack API Using the Heap**: Use the heap to implement the `stack_push` and `stack_pop` functions. The `order` field in the heap elements will ensure that the last pushed item is always popped first, adhering to the stack's LIFO principle.

Here's a simplified version in C:

```c
#include <stdio.h>
#include <stdlib.h>

typedef struct {
    int order;
    void* data;
} HeapItem;

typedef struct {
    HeapItem* items;
    int size;
    int capacity;
    int orderCounter;
} MaxHeap;

void initHeap(MaxHeap* heap, int capacity) {
    heap->items = malloc(sizeof(HeapItem) * capacity);
    heap->size = 0;
    heap->capacity = capacity;
    heap->orderCounter = 0;
}

void resizeHeapIfNeeded(MaxHeap* heap) {
    if (heap->size >= heap->capacity) {
        heap->capacity *= 2;
        heap->items = realloc(heap->items, sizeof(HeapItem) * heap->capacity);
    }
}

void swapHeapItems(HeapItem* a, HeapItem* b) {
    HeapItem temp = *a;
    *a = *b;
    *b = temp;
}

void siftUp(MaxHeap* heap, int index) {
    while (index != 0 && heap->items[(index - 1) / 2].order < heap->items[index].order) {
        swapHeapItems(&heap->items[index], &heap->items[(index - 1) / 2]);
        index = (index - 1) / 2;
    }
}

void heapPush(MaxHeap* heap, void* data) {
    resizeHeapIfNeeded(heap);
    heap->items[heap->size].data = data;
    heap->items[heap->size].order = heap->orderCounter++;
    siftUp(heap, heap->size);
    heap->size++;
}

void* heapPop(MaxHeap* heap) {
    if (heap->size == 0) {
        return NULL; // Heap is empty
    }

    void* result = heap->items[0].data;
    heap->items[0] = heap->items[--heap->size];
    // Implement siftDown here to maintain heap property
    return result;
}

// Example usage:
int main() {
    MaxHeap heap;
    initHeap(&heap, 10);

    int data1 = 1, data2 = 2, data3 = 3;
    heapPush(&heap, &data1);
    heapPush(&heap, &data2);
    heapPush(&heap, &data3);

    printf("%d\n", *(int*)heapPop(&heap)); // Expects 3
    printf("%d\n", *(int*)heapPop(&heap)); // Expects 2
    printf("%d\n", *(int*)heapPop(&heap)); // Expects 1

    free(heap.items);
    return 0;
}
```

This example demonstrates a basic max heap implementation in C, focusing on the concept rather than covering all edge cases and efficiency optimizations (like dynamic resizing or complete error handling). For a full-fledged application, you'd want to add more robust handling for edge cases, dynamic resizing of the heap, and possibly generic functions using `void*` pointers and function pointers for comparison to make the heap more versatile.

In [2]:
%%writefile stack_using_heap.c
#include <stdio.h>
#include <stdlib.h>

typedef struct {
    int order;
    void* data;
} HeapItem;

typedef struct {
    HeapItem* items;
    int size;
    int capacity;
    int orderCounter;
} MaxHeap;

void initHeap(MaxHeap* heap, int capacity) {
    heap->items = malloc(sizeof(HeapItem) * capacity);
    heap->size = 0;
    heap->capacity = capacity;
    heap->orderCounter = 0;
}

void resizeHeapIfNeeded(MaxHeap* heap) {
    if (heap->size >= heap->capacity) {
        heap->capacity *= 2;
        heap->items = realloc(heap->items, sizeof(HeapItem) * heap->capacity);
    }
}

void swapHeapItems(HeapItem* a, HeapItem* b) {
    HeapItem temp = *a;
    *a = *b;
    *b = temp;
}

void siftUp(MaxHeap* heap, int index) {
    while (index != 0 && heap->items[(index - 1) / 2].order < heap->items[index].order) {
        swapHeapItems(&heap->items[index], &heap->items[(index - 1) / 2]);
        index = (index - 1) / 2;
    }
}

void heapPush(MaxHeap* heap, void* data) {
    resizeHeapIfNeeded(heap);
    heap->items[heap->size].data = data;
    heap->items[heap->size].order = heap->orderCounter++;
    siftUp(heap, heap->size);
    heap->size++;
}

void* heapPop(MaxHeap* heap) {
    if (heap->size == 0) {
        return NULL; // Heap is empty
    }

    void* result = heap->items[0].data;
    heap->items[0] = heap->items[--heap->size];
    // Implement siftDown here to maintain heap property
    return result;
}

// Example usage:
int main() {
    MaxHeap heap;
    initHeap(&heap, 10);

    int data1 = 1, data2 = 2, data3 = 3;
    heapPush(&heap, &data1);
    heapPush(&heap, &data2);
    heapPush(&heap, &data3);

    printf("%d\n", *(int*)heapPop(&heap)); // Expects 3
    printf("%d\n", *(int*)heapPop(&heap)); // Expects 2
    printf("%d\n", *(int*)heapPop(&heap)); // Expects 1

    free(heap.items);
    return 0;
}



Writing stack_using_heap.c


In [3]:
!gcc stack_using_heap.c -o stack_using_heap


In [4]:
!./stack_using_heap


3
2
1
